In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

requests.packages.urllib3.disable_warnings()
url = 'http://books.toscrape.com/'
headers = {
'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'}
book = requests.get(url, headers = headers, verify=False).text
soup=BeautifulSoup(book, 'html.parser')

In [40]:
#first 20 book names
souppart=[]
twenty=[]
links=[]
from word2number import w2n
for i in soup.find_all('article'):
    souppart.append(i)
for i in souppart:
    temp=[]
    temp.append(i.h3.a.get('title'))
    temp.append(w2n.word_to_num(i.p.get('class')[1]))
    temp.append(i.select('p.price_color')[0].text[2:])
    links.append('http://books.toscrape.com/'+i.a.get('href'))
    twenty.append(temp)
df=pd.DataFrame(twenty, columns=['name', 'rating','price'])

In [70]:
pd.set_option('display.max_colwidth', 500)
df

,name,rating,price
0,A Light in the Attic,3,51.77
1,Tipping the Velvet,1,53.74
2,Soumission,1,50.10
3,Sharp Objects,4,47.82
4,Sapiens: A Brief History of Humankind,5,54.23
5,The Requiem Red,1,22.65
6,The Dirty Little Secrets of Getting Your Dream Job,4,33.34
7,"The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull",3,17.93
8,The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics,4,22.60
9,The Black Maria,1,52.15


In [74]:
import re
bookpage=[]
for i in links:
    booktemp=[]
    eachbook=requests.get(i, headers = headers, verify=False).text
    booksoup=BeautifulSoup(eachbook, 'html.parser')
    booktemp.append(booksoup.ul.select('li:nth-child(3)')[0].text.strip())
    booktemp.append(booksoup.td.text)
    booktemp.append(re.findall(r'\((\d+?)\b',booksoup.find('p',{'class':'instock availability'}).text.strip())[0])
   # print(re.findall(r'\((\d+?)\b',booksoup.find('p',{'class':'instock availability'}).text.strip()))
    bookpage.append(booktemp)


['22']
['20']
['20']
['20']
['20']
['19']
['19']
['19']
['19']
['19']
['19']
['19']
['19']
['19']
['19']
['19']
['19']
['19']
['19']
['19']


In [69]:
df2=pd.DataFrame(bookpage, columns=['Genre', 'UPC','Stock'])
dfs=pd.concat([df,df2], axis=1)
dfs

,name,rating,price,Genre,UPC,Stock
0,A Light in the Attic,3,51.77,Poetry,a897fe39b1053632,22
1,Tipping the Velvet,1,53.74,Historical Fiction,90fa61229261140a,20
2,Soumission,1,50.10,Fiction,6957f44c3847a760,20
3,Sharp Objects,4,47.82,Mystery,e00eb4fd7b871a48,20
4,Sapiens: A Brief History of Humankind,5,54.23,History,4165285e1663650f,20
5,The Requiem Red,1,22.65,Young Adult,f77dbf2323deb740,19
6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,Business,2597b5a345f45e1b,19
7,The Coming Woman: A Novel Based on the Life of...,3,17.93,Default,e72a5dfc7e9267b2,19
8,The Boys in the Boat: Nine Americans and Their...,4,22.60,Default,e10e1e165dc8be4a,19
9,The Black Maria,1,52.15,Poetry,1dfe412b8ac00530,19


In [76]:
dfs.to_csv('./bookscape.csv', index=False)